In [119]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import Series
from sklearn.feature_selection import f_classif, mutual_info_classif
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [120]:
rand=42

In [121]:
DATA_DIR = '/kaggle/input/sf-scoring/'
#DATA_DIR = './'
df_train = pd.read_csv(DATA_DIR +'/train.csv')
df_test = pd.read_csv(DATA_DIR +'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [122]:
sample_submission.shape

In [123]:
df_test.shape

In [124]:
df_train.info()

In [125]:
df_train.head(5)

In [126]:
df_test.info()

In [127]:
sample_submission.head(5)

In [128]:
sample_submission.info()

In [129]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [130]:
# основные статистики
data.describe()

In [131]:
# посмотрим количество уникальных значений 
data.nunique(dropna=False)

In [132]:
# Построим диаграмму для переменной default
data['default'].value_counts(ascending=True).plot(kind='barh')

In [133]:
# удалим ненужные столбцы
data.drop(['client_id','app_date',], axis = 1, inplace=True)

In [134]:
# проверим на наличие пропусков
data.isnull().sum(axis=0)

In [135]:
data.education.value_counts().plot.bar()

In [136]:
# Заполним пропуски в столбце education наиболее частым значением 'SCH'
data['education'].fillna('SCH',inplace=True)

## Визуализация

In [137]:
#Построим графики распределения числовых переменных.
#числовые переменные
num_cols = ['age', 'score_bki', 'decline_app_cnt', 'bki_request_cnt', 'income']
#категориальные переменные
cat_cols = ['education', 'first_time', 'sna', 'work_address', 'home_address', 'region_rating']
#бинарные переменные
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

In [138]:
for i in num_cols:
    plt.figure()
    sns.distplot(data[i][data[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

In [139]:
# логарифмируем числовые признаки
data['age'] = np.log(data['age']+1)
data['decline_app_cnt'] = np.log(data['decline_app_cnt']+1)
data['income'] = np.log(data['income']+1)
data['bki_request_cnt'] = np.log(data['bki_request_cnt']+1)

In [140]:
#Нормализация численных данных
from sklearn.preprocessing import StandardScaler

for column in num_cols:
    data[column] = StandardScaler().fit_transform(np.array(data[column].values).reshape(-1, 1))

Построим боксплоты для числовых переменных

In [141]:
sns.boxplot(x='default',y='age',data=data)

Дефолтные клиенты в среднем младше

In [142]:
sns.boxplot(x='default',y='bki_request_cnt',data=data)

Дефолтные клиенты в среднем имеют больше запросов в БКИ

In [143]:
sns.boxplot(x='default',y='income',data=data)

In [144]:
data.groupby('default').income.mean()

Дефолтные клиенты в среднем имеют более низкий доход

In [145]:
data.head()

## Значимость непрерывных переменных

In [146]:
#7. Значимость непрерывных переменных
#В основе метода оценки значимости переменных лежит однофакторный дисперсионный анализ (ANOVA). 
#Основу процедуры составляет обобщение результатов двух выборочных t-тестов для независимых выборок (2-sample t). 
#В качестве меры значимости мы будем использовать значение f-статистики. 
#Чем значение статистики выше, тем меньше вероятность того, что средние значения не отличаются, 
#и тем важнее данный признак для нашей линейной модели.
#числовые переменные

imp_num = pd.Series(f_classif(data[num_cols], data['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [147]:
# Заменим значения в столбце education на dummie-переменные
data = pd.get_dummies(data, columns=['education'], dummy_na=False)

In [148]:
# применим  LabelEncoder() для преобразования бинарных и категориальных переменных в цифровой формат
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for column in bin_cols:
    data[column] = le.fit_transform(data[column])
    
#columns = ['first_time', 'sna', 'work_address', 'home_address', 'region_rating']

#for column in columns:
#    data[column] = le.fit_transform(data[column])

In [149]:
data.info()

In [150]:
data

In [151]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)

y = train_data['default'].values  # наш таргет
х = train_data.drop(['default'], axis=1)

In [152]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
from sklearn.model_selection import train_test_split

# выделим 20% данных на валидацию (параметр test_size)
х_train, х_test, y_train, y_test = train_test_split(х, y, test_size=0.2, stratify=y, shuffle=True, random_state=rand)

In [153]:
# проверяем
х_train.shape, х_test.shape, y_train.shape, y_test.shape

In [154]:
# Импортируем необходимые библиотеки:
from sklearn.linear_model import LogisticRegression # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели
from sklearn.model_selection import GridSearchCV

In [155]:
model = LogisticRegression(tol=0.001,solver='newton-cg', max_iter=50, class_weight='balanced', random_state=rand)
model.fit(х_train, y_train)
y_pred = model.predict(х_test)

In [156]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']
classification_report = classification_report(y_test, y_pred, target_names=target_names)
print(classification_report)

In [157]:
# Отрисовать ROC кривую
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score,confusion_matrix
from matplotlib import pyplot as plt

# Посчитать значения ROC кривой и значение площади под кривой AUC
fpr, tpr, thresholds = roc_curve(y_train,model.predict_proba(х_train).T[1])
roc_auc = roc_auc_score(y_train,model.predict_proba(х_train).T[1])   
plt.figure()
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
plt.title('Logistic Regression ROC AUC = %0.4f' % roc_auc)
plt.xlabel('False positive rate (FPR)')
plt.ylabel('True positive rate (TPR)')
plt.legend(loc = 'lower right')

In [158]:
#Построение матрицы ошибок
print('confusion_matrix:')
print(confusion_matrix(y_test,y_pred))

In [159]:
#обучаем финальную модель на всех обучающи данных
model_final = LogisticRegression(tol=0.001,solver='newton-cg', max_iter=50, class_weight='balanced', random_state=rand)
model_final.fit(х, y)

In [160]:
predict_submission = model_final.predict(test_data)

In [161]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [162]:
sample_submission.describe()

In [163]:
!kaggle competitions submit -c sf-scoring -f ssubmission.csv -m "Message"
# !kaggle competitions submit your-competition-name -f submission.csv -m 'My submission message'